# Demonstartion of Reject Option Classification with AIF360
- The first part is to show the AIF360 Reject Option Classification algorithm
- The second part is to show the AIF360 ROC algorithm with FairFace dataset

# Part 1: AIF360 Reject Option Classification (AIF360 Dataset)

#### This notebook demonstrates the use of the Reject Option Classification (ROC) post-processing algorithm for bias mitigation.
- The debiasing function used is implemented in the `RejectOptionClassification` class.
- Divide the dataset into training, validation, and testing partitions.
- Train classifier on original training data.
- Estimate the optimal classification threshold, that maximizes balanced accuracy without fairness constraints.
- Estimate the optimal classification threshold, and the critical region boundary (ROC margin) using a validation set for the desired constraint on fairness. The best parameters are those that maximize the classification threshold while satisfying the fairness constraints.
- The constraints can be used on the following fairness measures:
    * Statistical parity difference on the predictions of the classifier
    * Average odds difference for the classifier
    * Equal opportunity difference for the classifier
- Determine the prediction scores for testing data. Using the estimated optimal classification threshold, compute accuracy and fairness metrics.
- Using the determined optimal classification threshold and the ROC margin, adjust the predictions. Report accuracy and fairness metric on the new predictions.

source: https://github.com/Trusted-AI/AIF360/blob/main/examples/demo_reject_option_classification.ipynb

In [282]:
# pip install git+https://github.com/Trusted-AI/AIF360


In [283]:
# %pip install aif360


In [284]:
# %pip install common-utils


In [ ]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from warnings import warn



from aif360.datasets import StandardDataset
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification
from aif360.algorithms.postprocessing import RejectOptionClassification
from aif360.detectors.mdss.generator import get_random_subset

from aif360.sklearn.datasets import fetch_adult
#from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider


In [286]:
# %pip install aif360[Reductions]


In [287]:
# Metrics function
from collections import OrderedDict
from aif360.metrics import ClassificationMetric


def compute_metrics(dataset_true, dataset_pred,
                    unprivileged_groups, privileged_groups,
                    disp=True):
    """ Compute the key metrics """
    classified_metric_pred = ClassificationMetric(dataset_true,
                                                  dataset_pred,
                                                  unprivileged_groups=unprivileged_groups,
                                                  privileged_groups=privileged_groups)
    metrics = OrderedDict()
    metrics["Balanced accuracy"] = 0.5*(classified_metric_pred.true_positive_rate() +
                                        classified_metric_pred.true_negative_rate())
    metrics["Statistical parity difference"] = classified_metric_pred.statistical_parity_difference()
    metrics["Disparate impact"] = classified_metric_pred.disparate_impact()
    metrics["Average odds difference"] = classified_metric_pred.average_odds_difference()
    metrics["Equal opportunity difference"] = classified_metric_pred.equal_opportunity_difference()
    metrics["Theil index"] = classified_metric_pred.theil_index()

    if disp:
        for k in metrics:
            print("%s = %.4f" % (k, metrics[k]))

    return metrics


#### Load dataset and specify options

In [ ]:
import os
import urllib.request

urls = {
    "adult.data": "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    "adult.test": "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
    "adult.names": "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names"
}

# Replace the path with your own path to aif360 direcotry
# my_data_folder = os.path.join(
#     r"c:\Users\your_directory\anaconda3\envs\pymc_env\Lib\site-packages\aif360\data\raw\adult")

for filename, url in urls.items():
    file_path = os.path.join(my_data_folder, filename)
    urllib.request.urlretrieve(url, file_path)
    print(f"Downloaded {filename} to {file_path}")


Downloaded adult.data to c:\Users\n_oha\anaconda3\envs\pymc_env\Lib\site-packages\aif360\data\raw\adult\adult.data
Downloaded adult.test to c:\Users\n_oha\anaconda3\envs\pymc_env\Lib\site-packages\aif360\data\raw\adult\adult.test
Downloaded adult.names to c:\Users\n_oha\anaconda3\envs\pymc_env\Lib\site-packages\aif360\data\raw\adult\adult.names


In [290]:
# import dataset
dataset_used = "adult"  # "adult", "german", "compas"
protected_attribute_used = 1  # 1, 2

if dataset_used == "adult":
    #     dataset_orig = AdultDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
        dataset_orig = load_preproc_data_adult(['sex'])
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]
        dataset_orig = load_preproc_data_adult(['race'])

elif dataset_used == "german":
    #     dataset_orig = GermanDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
        dataset_orig = load_preproc_data_german(['sex'])
    else:
        privileged_groups = [{'age': 1}]
        unprivileged_groups = [{'age': 0}]
        dataset_orig = load_preproc_data_german(['age'])

elif dataset_used == "compas":
    #     dataset_orig = CompasDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
        dataset_orig = load_preproc_data_compas(['sex'])
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]
        dataset_orig = load_preproc_data_compas(['race'])


# Metric used (should be one of allowed_metrics)
metric_name = "Statistical parity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05

# random seed for calibrated equal odds prediction
np.random.seed(1)

# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")


#### Split into train, test and validation

In [291]:
# Get the dataset and split into train and test
dataset_orig_train, dataset_orig_vt = dataset_orig.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split(
    [0.5], shuffle=True)


#### Clean up training data and display properties of the data

In [292]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes,
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)


#### Training Dataset shape

(34189, 18)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['sex']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['race', 'sex', 'Age (decade)=10', 'Age (decade)=20', 'Age (decade)=30', 'Age (decade)=40', 'Age (decade)=50', 'Age (decade)=60', 'Age (decade)=>=70', 'Education Years=6', 'Education Years=7', 'Education Years=8', 'Education Years=9', 'Education Years=10', 'Education Years=11', 'Education Years=12', 'Education Years=<6', 'Education Years=>12']


#### Metric for original training data

In [293]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" %
      metric_orig_train.mean_difference())


#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.190244


### Train classifier on original data

In [294]:
# Logistic regression classifier and predictions
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()

lmod = LogisticRegression()
lmod.fit(X_train, y_train)
y_train_pred = lmod.predict(X_train)

# positive class index
pos_ind = np.where(lmod.classes_ == dataset_orig_train.favorable_label)[0][0]

dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_train_pred.labels = y_train_pred


#### Obtain scores for validation and test sets

In [295]:
dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
X_valid = scale_orig.transform(dataset_orig_valid_pred.features)
y_valid = dataset_orig_valid_pred.labels
dataset_orig_valid_pred.scores = lmod.predict_proba(
    X_valid)[:, pos_ind].reshape(-1, 1)

dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
X_test = scale_orig.transform(dataset_orig_test_pred.features)
y_test = dataset_orig_test_pred.labels
dataset_orig_test_pred.scores = lmod.predict_proba(
    X_test)[:, pos_ind].reshape(-1, 1)


### Find the optimal parameters from the validation set

#### Best threshold for classification only (no fairness)

In [296]:
num_thresh = 100
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):

    fav_inds = dataset_orig_valid_pred.scores > class_thresh
    dataset_orig_valid_pred.labels[fav_inds] = dataset_orig_valid_pred.favorable_label
    dataset_orig_valid_pred.labels[~fav_inds] = dataset_orig_valid_pred.unfavorable_label

    classified_metric_orig_valid = ClassificationMetric(dataset_orig_valid,
                                                        dataset_orig_valid_pred,
                                                        unprivileged_groups=unprivileged_groups,
                                                        privileged_groups=privileged_groups)

    ba_arr[idx] = 0.5*(classified_metric_orig_valid.true_positive_rate()
                       + classified_metric_orig_valid.true_negative_rate())

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
print("Optimal classification threshold (no fairness constraints) = %.4f" %
      best_class_thresh)


Best balanced accuracy (no fairness constraints) = 0.7463
Optimal classification threshold (no fairness constraints) = 0.2872


#### Estimate optimal parameters for the ROC method

In [297]:
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups,
                                 privileged_groups=privileged_groups,
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                 num_class_thresh=100, num_ROC_margin=50,
                                 metric_name=metric_name,
                                 metric_ub=metric_ub, metric_lb=metric_lb)
ROC = ROC.fit(dataset_orig_valid, dataset_orig_valid_pred)


In [298]:
print("Optimal classification threshold (with fairness constraints) = %.4f" %
      ROC.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC.ROC_margin)


Optimal classification threshold (with fairness constraints) = 0.1981
Optimal ROC margin = 0.1011


### Predictions from Validation Set

In [299]:
# Metrics for the test set
fav_inds = dataset_orig_valid_pred.scores > best_class_thresh
dataset_orig_valid_pred.labels[fav_inds] = dataset_orig_valid_pred.favorable_label
dataset_orig_valid_pred.labels[~fav_inds] = dataset_orig_valid_pred.unfavorable_label

display(Markdown("#### Validation set"))
display(Markdown(
    "##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

metric_valid_bef = compute_metrics(dataset_orig_valid, dataset_orig_valid_pred,
                                   unprivileged_groups, privileged_groups)


#### Validation set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Balanced accuracy = 0.7463
Statistical parity difference = -0.3670
Disparate impact = 0.2744
Average odds difference = -0.3140
Equal opportunity difference = -0.3666
Theil index = 0.1113


In [300]:
# Transform the validation set
dataset_transf_valid_pred = ROC.predict(dataset_orig_valid_pred)

display(Markdown("#### Validation set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_valid_aft = compute_metrics(dataset_orig_valid, dataset_transf_valid_pred,
                                   unprivileged_groups, privileged_groups)


#### Validation set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 0.7090
Statistical parity difference = -0.0454
Disparate impact = 0.8996
Average odds difference = 0.0363
Equal opportunity difference = 0.0197
Theil index = 0.1172


### Predictions from Test Set

In [242]:
# Metrics for the test set
fav_inds = dataset_orig_test_pred.scores > best_class_thresh
dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label

display(Markdown("#### Test set"))
display(Markdown(
    "##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred,
                                  unprivileged_groups, privileged_groups)


#### Test set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Balanced accuracy = 0.7437
Statistical parity difference = -0.3580
Disparate impact = 0.2794
Average odds difference = -0.3181
Equal opportunity difference = -0.3769
Theil index = 0.1129


In [243]:
# Metrics for the transformed test set
dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)

display(Markdown("#### Test set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred,
                                  unprivileged_groups, privileged_groups)


#### Test set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 0.7141
Statistical parity difference = -0.0402
Disparate impact = 0.9088
Average odds difference = 0.0423
Equal opportunity difference = 0.0407
Theil index = 0.1171


## Summary of Optimal Parameters
We show the optimal parameters for all combinations of metrics optimized, datasets, and protected attributes below.

### Fairness Metric: Statistical parity difference, Accuracy Metric: Balanced accuracy

#### Performance

| Dataset |Sex (Acc-Bef)|Sex (Acc-Aft)|Sex (Fair-Bef)|Sex (Fair-Aft)|Race/Age (Acc-Bef)|Race/Age (Acc-Aft)|Race/Age (Fair-Bef)|Race/Age (Fair-Aft)|
|-|-|-|-|-|-|-|-|-|
|Adult (Valid)|0.7473|0.6051|-0.3703|-0.0436|0.7473|0.6198|-0.2226|-0.0007|
|Adult (Test)|0.7417|0.5968|-0.3576|-0.0340|0.7417|0.6202|-0.2279|0.0006|
|German (Valid)|0.6930|0.6991|-0.0613|0.0429|0.6930|0.6607|-0.2525|-0.0328|
|German (Test)|0.6524|0.6460|-0.0025|0.0410|0.6524|0.6317|-0.3231|-0.1038|
|Compas (Valid)|0.6599|0.6400|-0.2802|0.0234|0.6599|0.6646|-0.3225|-0.0471|
|Compas (Test)|0.6774|0.6746|-0.2724|-0.0313|0.6774|0.6512|-0.2494|0.0578|

#### Optimal Parameters

| Dataset |Sex (Class. thresh.)|Sex (Class. thresh. - fairness)|Sex (ROC margin - fairness)| Race/Age (Class. thresh.)|Race/Age (Class. thresh. - fairness)|Race/Age (ROC margin - fairness)|
|-|-|-|-|-|-|-|
|Adult|0.2674|0.5049|0.1819|0.2674|0.5049|0.0808|
|German|0.6732|0.6237|0.0538|0.6732|0.7029|0.0728|
|Compas|0.5148|0.5841|0.0679|0.5148|0.5841|0.0679|

### Fairness Metric: Average odds difference, Accuracy Metric: Balanced accuracy

#### Performance

| Dataset |Sex (Acc-Bef)|Sex (Acc-Aft)|Sex (Fair-Bef)|Sex (Fair-Aft)|Race/Age (Acc-Bef)|Race/Age (Acc-Aft)|Race/Age (Fair-Bef)|Race/Age (Fair-Aft)|
|-|-|-|-|-|-|-|-|-|
|Adult (Valid)|0.7473|0.6058|-0.2910|-0.0385|0.7473|0.6593|-0.1947|-0.0444|
|Adult (Test)|0.7417|0.6024|-0.3281|-0.0438|0.7417|0.6611|-0.1991|-0.0121|
|German (Valid)|0.6930|0.6930|-0.0039|-0.0039|0.6930|0.6807|-0.0919|-0.0193|
|German (Test)|0.6524|0.6571|0.0071|0.0237|0.6524|0.6587|-0.3278|-0.2708|
|Compas (Valid)|0.6599|0.6416|-0.2285|-0.0332|0.6599|0.6646|-0.2918|-0.0105|
|Compas (Test)|0.6774|0.6721|-0.2439|-0.0716|0.6774|0.6512|-0.1927|0.1145|

#### Optimal Parameters

| Dataset |Sex (Class. thresh.)|Sex (Class. thresh. - fairness)|Sex (ROC margin - fairness)| Race/Age (Class. thresh.)|Race/Age (Class. thresh. - fairness)|Race/Age (ROC margin - fairness)|
|-|-|-|-|-|-|-|
|Adult|0.2674|0.5049|0.1212|0.2674|0.5049|0.0505|
|German|0.6732|0.6633|0.0137|0.6732|0.6732|0.0467|
|Compas|0.5148|0.5742|0.0608|0.5148|0.5841|0.0679|


### Fairness Metric: Equal opportunity difference, Accuracy Metric: Balanced accuracy

#### Performance

| Dataset |Sex (Acc-Bef)|Sex (Acc-Aft)|Sex (Fair-Bef)|Sex (Fair-Aft)|Race/Age (Acc-Bef)|Race/Age (Acc-Aft)|Race/Age (Fair-Bef)|Race/Age (Fair-Aft)|
|-|-|-|-|-|-|-|-|-|
|Adult (Valid)|0.7473|0.6051|-0.3066|-0.0136|0.7473|0.6198|-0.2285|0.0287|
|Adult (Test)|0.7417|0.5968|-0.4001|-0.0415|0.7417|0.6202|-0.2165|0.1193|
|German (Valid)|0.6930|0.6930|-0.0347|-0.0347|0.6930|0.6597|0.1162|-0.0210|
|German (Test)|0.6524|0.6571|0.0400|0.0733|0.6524|0.6190|-0.3556|-0.4333|
|Compas (Valid)|0.6599|0.6416|-0.1938|0.0244|0.6599|0.6646|-0.2315|0.0002|
|Compas (Test)|0.6774|0.6721|-0.1392|0.0236|0.6774|0.6512|-0.1877|0.1196|

#### Optimal Parameters

| Dataset |Sex (Class. thresh.)|Sex (Class. thresh. - fairness)|Sex (ROC margin - fairness)| Race/Age (Class. thresh.)|Race/Age (Class. thresh. - fairness)|Race/Age (ROC margin - fairness)|
|-|-|-|-|-|-|-|
|Adult|0.2674|0.5049|0.1819|0.2674|0.5049|0.0808|
|German|0.6732|0.6633|0.0137|0.6732|0.6039|0.0000|
|Compas|0.5148|0.5742|0.0608|0.5148|0.5841|0.0679|


# Part 2: AIF360 Reject Option Classification (FairFace Dataset)
Source: https://medium.com/@james.irving.phd/blog-post-series-ai-fairness-360-mitigating-bias-in-machine-learning-models-c1ec744c91c4

## Apply FairFace Test Dataset

In [244]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request
import importlib.util
import sys
import os
import io
import requests
from sklearn.preprocessing import LabelEncoder


### Prepare FairFace dataset

In [245]:
# Replace the test set with the FairFace test set

df_train = pd.read_csv('fairface_label_train.csv')
df_train['split'] = 'train'
df_test = pd.read_csv('fairface_label_val.csv')
df_test['split'] = 'test'
df = pd.concat([df_train, df_test])

df = pd.read_csv('fairface_label_val.csv')
df = df.drop(columns=['file'])
df.columns.values.tolist()  # to see all the variables


['age', 'gender', 'race', 'service_test']

In [246]:
df.rename(columns={'gender':'Sex'}, inplace=True)
df.rename(columns={'race':'Race'}, inplace=True)

df.columns.values.tolist()  # to see all the variables


['age', 'Sex', 'Race', 'service_test']

In [247]:
# Filter service_test == True

df = df[df['service_test'] == True]
df.columns


Index(['age', 'Sex', 'Race', 'service_test'], dtype='object')

In [248]:
# Encode the 'race' column as binary white or non-white.
Race_map = {'East Asian': 1, 'White': 0, 'Latino_Hispanic': 1,
            'Southeast Asian': 1, 'Black': 1, 'Indian': 1, 'Middle Eastern': 1}
df['Race'] = df['Race'].map(Race_map)

# Encode the "gender" column
Sex_map = {"Male": 0, "Female":1}
df['Sex'] = df['Sex'].map(Sex_map)


df['Race'].unique(), df['Sex'].unique()


(array([1, 0], dtype=int64), array([1, 0], dtype=int64))

In [249]:
# Saving the protcted attribute mapping dictionaries to a dict
protected_attribute_maps = {"Race": Race_map,
                            "Sex": Sex_map}


In [250]:
# Impute NaNs with most frequent value
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline

# Enable pandas dataframe output
from sklearn import set_config
set_config(transform_output='pandas')


In [251]:
# Categorical Pipeline
cat_cols = df.select_dtypes(include='object').columns
cat_imputer = SimpleImputer(strategy='most_frequent')
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
cat_pipe = make_pipeline(cat_imputer, ohe)

# Numeric Pipeline
num_cols = df.select_dtypes(include='number').columns
num_imputer = SimpleImputer(strategy='mean')
num_pipe = make_pipeline(num_imputer)

# Convert Boolean Columns to Integers
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)


# Create the column Transformer
preprocessor = ColumnTransformer(transformers=[('cat', cat_pipe, cat_cols),
                                               ('num', num_pipe, num_cols)],
                                 remainder='passthrough',
                                 verbose_feature_names_out=False)

preprocessor


ColumnTransformer(remainder='passthrough',
                  transformers=[('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['age'], dtype='object')),
                                ('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer())]),
                                 Index(['Sex', 'Race'], dtype='object'))],
                  verbose_feature_names_out=False)

In [252]:
# Fit and Transform the data
final_df = preprocessor.fit_transform(df)
final_df.head()  # Display the first few rows of the transformed data


,age_0-2,age_10-19,age_20-29,age_3-9,age_30-39,age_40-49,age_50-59,age_60-69,age_more than 70,Sex,Race,service_test
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1
6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1


In [253]:
## Saving the protcted attribute mapping dictionaries to a dict

#Commented Out: previously saved above
# protected_attribute_maps = {"Race":race_map,
#                             "Sex":sex_map}

print("- protected_attribute_maps:")
display(protected_attribute_maps)

# Manually defining the list of privileged group NAMES to a dict
prot_attrs_priv_group_names = {'Race':["White"],
                               "Sex":["Male"]}

print("- prot_attrs_priv_group_names:")
prot_attrs_priv_group_names


- protected_attribute_maps:


{'Race': {'East Asian': 1,
  'White': 0,
  'Latino_Hispanic': 1,
  'Southeast Asian': 1,
  'Black': 1,
  'Indian': 1,
  'Middle Eastern': 1},
 'Sex': {'Male': 0, 'Female': 1}}

- prot_attrs_priv_group_names:


{'Race': ['White'], 'Sex': ['Male']}

In [254]:
# Creating a data dictionary like the one used in AIF360's medical expenditure example
target = 'East Asian'  # The target variable for classification

# Ensure the target column exists in final_df before dropping it
if target not in final_df.columns:
    final_df[target] = np.random.choice([0, 1], size=len(final_df))  # Add the target column if missing

# Saving the feature names as a list
feature_names = final_df.drop(columns=target).columns
protected_attributes_names = list(protected_attribute_maps.keys())

# Starting our own data dictionary
DATA_DICT = dict(feature_names=feature_names, # list of feature columns
                 label_names = [target], # list with just the target label column
                 protected_attributes_names = protected_attributes_names, # list of protected attribute columns
                 
                # Saving information about each row (not used )
                #  instance_names = final_df.index, # list of each row's name
                #  instance_weights = np.ones_like(final_df.index), # list of each row's weight (1.0 for now
                
                
                # List of arrays of privileged group numbers: will be filled in later
                privileged_protected_attributes = [], 
                # List of arrays of unprivileged group numers: Will be filled in later
                unprivileged_protected_attributes = [], 
)
                 
DATA_DICT.keys()


dict_keys(['feature_names', 'label_names', 'protected_attributes_names', 'privileged_protected_attributes', 'unprivileged_protected_attributes'])

In [255]:
# Numeric index of current protected attribute
attr_idx = 0

# slice the name of the current protected attribute
attr_name = protected_attributes_names[attr_idx]

# Get the mapping for the current protected attribute
attr_map = protected_attribute_maps[attr_name]
attr_map


{'East Asian': 1,
 'White': 0,
 'Latino_Hispanic': 1,
 'Southeast Asian': 1,
 'Black': 1,
 'Indian': 1,
 'Middle Eastern': 1}

In [256]:
# # Get the privileged group names for the current protected attribute
priv_group_names = prot_attrs_priv_group_names[attr_name]
priv_group_names


['White']

In [257]:
# Get the privileged values for the current protected attribute
privileged_group_nums = np.array([attr_map[pg] for pg in priv_group_names])
privileged_group_nums


array([0])

In [258]:
# Get the unprivileged values for the current protected attribute
unprivileged_group_nums = np.array(
    [v for v in attr_map.values() if v not in privileged_group_nums])
unprivileged_group_nums


array([1, 1, 1, 1, 1, 1])

In [259]:
# Saving the privileged and unprivileged group values to the data dictionary
DATA_DICT['privileged_protected_attributes'].append(privileged_group_nums)
DATA_DICT['unprivileged_protected_attributes'].append(unprivileged_group_nums)


## Converting the dataframe to AIF360 Dataset

In [260]:
# Reviewing the structure of our DATA_DICT
DATA_DICT.keys()


dict_keys(['feature_names', 'label_names', 'protected_attributes_names', 'privileged_protected_attributes', 'unprivileged_protected_attributes'])

In [261]:
from aif360.datasets import BinaryLabelDataset

# Add the target column 'Reincarcerated' to final_df
# For demonstration, we assign random binary values (0 or 1) as the target.
# Replace this logic with your actual target column values.
#final_df['Reincarcerated'] = np.random.choice([0, 1], size=len(final_df))

# Update the BinaryLabelDataset creation
binary_dataset = BinaryLabelDataset(df=final_df,
                                    label_names=DATA_DICT['label_names'],
                                    protected_attribute_names=DATA_DICT['protected_attributes_names'],
                                    favorable_label=0,  # Non-recidivism is favorable
                                    unfavorable_label=1  # Recidivism is unfavorable
                                    )
binary_dataset


               instance weights features                              \
                                                                       
                                 age_0-2 age_10-19 age_20-29 age_3-9   
instance names                                                         
1                           1.0      0.0       0.0       0.0     0.0   
2                           1.0      0.0       0.0       0.0     0.0   
3                           1.0      0.0       0.0       1.0     0.0   
6                           1.0      0.0       0.0       1.0     0.0   
10                          1.0      0.0       0.0       0.0     0.0   
...                         ...      ...       ...       ...     ...   
10939                       1.0      0.0       0.0       1.0     0.0   
10941                       1.0      0.0       0.0       0.0     1.0   
10947                       1.0      0.0       1.0       0.0     0.0   
10949                       1.0      0.0       0.0       0.0    

In [262]:
DATA_DICT['unprivileged_protected_attributes'][0]


array([1, 1, 1, 1, 1, 1])

In [263]:
DATA_DICT['protected_attributes_names'], DATA_DICT['privileged_protected_attributes'], DATA_DICT['unprivileged_protected_attributes']


(['Race', 'Sex'], [array([0])], [array([1, 1, 1, 1, 1, 1])])

In [264]:
# TEMP  testing of args - single dict per attribute

# For loop (based on tutorial list comp) to create a list of dictionaries 
# for unprivileged and privileged groups for each of the protected attributes
unprivileged_groups = []
privileged_groups = []

# Ensure the loop does not exceed the length of DATA_DICT['unprivileged_protected_attributes']
num_attributes = min(len(protected_attributes_names), len(DATA_DICT['unprivileged_protected_attributes']))
num_attributes = min(len(protected_attributes_names), len(
    DATA_DICT['privileged_protected_attributes']))

for sens_ind in range(num_attributes):
    # Saving the name of the current protected attribute
    sens_attr_name = protected_attributes_names[sens_ind]
    
    # Save the unique integer values for the current protected attribute
    unprivileged_groups.append({sens_attr_name: np.unique(DATA_DICT['unprivileged_protected_attributes'][sens_ind])})
    privileged_groups.append({sens_attr_name:  np.unique(DATA_DICT['privileged_protected_attributes'][sens_ind])})

unprivileged_groups, privileged_groups


([{'Race': array([1])}], [{'Race': array([0])}])

BinaryLabelDatasetMetric class must be instantiated for 1 protected attribute at a time (e.g. Sex or Race, but not both).

In [265]:
from aif360.explainers import Explainer
from aif360.metrics import Metric


class MetricTextExplainer(Explainer):
    """Class for explaining metric values with text.

    These briefly explain what a metric is and/or how it is calculated unless it
    is obvious (e.g. accuracy) and print the value.

    This class contains text explanations for all metric values regardless of
    which subclass they appear in. This will raise an error if the metric does
    not apply (e.g. calling `true_positive_rate` if
    `type(metric) == DatasetMetric`).
    """

    def __init__(self, metric):
        """Initialize a `MetricExplainer` object.

        Args:
            metric (Metric): The metric to be explained.
        """
        if isinstance(metric, Metric):
            self.metric = metric
        else:
            raise TypeError("metric must be a Metric.")

    def accuracy(self, privileged=None):
        if privileged is None:
            return "Classification accuracy (ACC): {}".format(
                self.metric.accuracy(privileged=privileged))
        return "Classification accuracy on {} instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.accuracy(privileged=privileged))

    def average_abs_odds_difference(self):
        return ("Average absolute odds difference (average of abs(TPR "
                "difference) and abs(FPR difference)): {}".format(
                    self.metric.average_abs_odds_difference()))

    def average_odds_difference(self):
        return ("Average odds difference (average of TPR difference and FPR "
                "difference, 0 = equality of odds): {}".format(
                    self.metric.average_odds_difference()))

    def between_all_groups_coefficient_of_variation(self):
        return "Between-group coefficient of variation: {}".format(
            self.metric.between_all_groups_coefficient_of_variation())

    def between_all_groups_generalized_entropy_index(self, alpha=2):
        return "Between-group generalized entropy index: {}".format(
            self.metric.between_all_groups_generalized_entropy_index(alpha=alpha))

    def between_all_groups_theil_index(self):
        return "Between-group Theil index: {}".format(
            self.metric.between_all_groups_theil_index())

    def between_group_coefficient_of_variation(self):
        return "Between-group coefficient of variation: {}".format(
            self.metric.between_group_coefficient_of_variation())

    def between_group_generalized_entropy_index(self, alpha=2):
        return "Between-group generalized entropy index: {}".format(
            self.metric.between_group_generalized_entropy_index(alpha=alpha))

    def between_group_theil_index(self):
        return "Between-group Theil index: {}".format(
            self.metric.between_group_theil_index())

    def coefficient_of_variation(self):
        return "Coefficient of variation: {}".format(
            self.metric.coefficient_of_variation())

    def consistency(self, n_neighbors=5):
        return "Consistency (Zemel, et al. 2013): {}".format(
            self.metric.consistency(n_neighbors=n_neighbors))

    def disparate_impact(self):
        return ("Disparate impact (probability of favorable outcome for "
                "unprivileged instances / probability of favorable outcome for "
                "privileged instances): {}".format(
                    self.metric.disparate_impact()))

    def error_rate(self, privileged=None):
        if privileged is None:
            return "Error rate (ERR = 1 - ACC): {}".format(
                self.metric.error_rate(privileged=privileged))
        return "Error rate on {} instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.error_rate(privileged))

    def error_rate_difference(self):
        return ("Error rate difference (error rate on unprivileged instances - "
                "error rate on privileged instances): {}".format(
                    self.metric.error_rate_difference()))

    def error_rate_ratio(self):
        return ("Error rate ratio (error rate on unprivileged instances / "
                "error rate on privileged instances): {}".format(
                    self.metric.error_rate_ratio()))

    def false_discovery_rate(self, privileged=None):
        if privileged is None:
            return "False discovery rate (FDR = FP / (FP + TP)): {}".format(
                self.metric.false_discovery_rate(privileged=privileged))
        return "False discovery rate on {} instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.false_discovery_rate(privileged=privileged))

    def false_discovery_rate_difference(self):
        return ("False discovery rate difference (false discovery rate on "
                "unprivileged instances - false discovery rate on privileged "
                "instances): {}".format(
                    self.metric.false_discovery_rate_difference()))

    def false_discovery_rate_ratio(self):
        return ("False discovery rate ratio (false discovery rate on "
                "unprivileged instances - false discovery rate on privileged "
                "instances): {}".format(
                    self.metric.false_discovery_rate_ratio()))

    def false_negative_rate(self, privileged=None):
        if privileged is None:
            return "False negative rate (FNR = FN / (TP + FN)): {}".format(
                self.metric.false_negative_rate(privileged=privileged))
        return "False negative rate on {} instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.false_negative_rate(privileged=privileged))

    def false_negative_rate_difference(self):
        return ("False negative rate difference (false negative rate on "
                "unprivileged instances - false negative rate on privileged "
                "instances): {}".format(
                    self.metric.false_negative_rate_difference()))

    def false_negative_rate_ratio(self):
        return ("False negative rate ratio (false negative rate on "
                "unprivileged instances / false negative rate on privileged "
                "instances): {}".format(
                    self.metric.false_negative_rate_ratio()))

    def false_omission_rate(self, privileged=None):
        if privileged is None:
            return "False omission rate (FOR = FN / (FN + TN)): {}".format(
                self.metric.false_omission_rate(privileged=privileged))
        return "False omission rate on {} instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.false_omission_rate(privileged=privileged))

    def falses_omission_rate_difference(self):
        return ("False omission rate difference (falses omission rate on "
                "unprivileged instances - falses omission rate on privileged "
                "instances): {}".format(
                    self.metric.falses_omission_rate_difference()))

    def false_omission_rate_ratio(self):
        return ("False omission rate ratio (false omission rate on "
                "unprivileged instances - false omission rate on privileged "
                "instances): {}".format(
                    self.metric.false_omission_rate_ratio()))

    def false_positive_rate(self, privileged=None):
        if privileged is None:
            return "False positive rate (FPR = FP / (FP + TN)): {}".format(
                self.metric.false_positive_rate(privileged=privileged))
        return "False positive rate on {} instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.false_positive_rate(privileged=privileged))

    def false_positive_rate_difference(self):
        return ("False positive rate difference (false positive rate on "
                "unprivileged instances - false positive rate on privileged "
                "instances): {}".format(
                    self.metric.false_positive_rate_difference()))

    def false_positive_rate_ratio(self):
        return ("False positive rate ratio (false positive rate on "
                "unprivileged instances / false positive rate on privileged "
                "instances): {}".format(
                    self.metric.false_positive_rate_ratio()))

    def generalized_entropy_index(self, alpha=2):
        return "Generalized entropy index (GE(alpha)): {}".format(
            self.metric.generalized_entropy_index(alpha=alpha))

    def mean_difference(self):
        return ("Mean difference (mean label value on unprivileged instances - "
                "mean label value on privileged instances): {}".format(
                    self.metric.mean_difference()))

    def negative_predictive_value(self, privileged=None):
        if privileged is None:
            return "Negative predictive value (NPV = TN / (TN + FN)): {}".format(
                self.metric.negative_predictive_value(privileged=privileged))
        return "Negative predictive value on {} instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.negative_predictive_value(privileged=privileged))

    def num_false_negatives(self, privileged=None):
        if privileged is None:
            return "Number of false negative instances (FN): {}".format(
                self.metric.num_false_negatives(privileged=privileged))
        return "Number of {} false negative instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.num_false_negatives(privileged=privileged))

    def num_false_positives(self, privileged=None):
        if privileged is None:
            return "Number of false positive instances (FP): {}".format(
                self.metric.num_false_positives(privileged=privileged))
        return "Number of {} false positive instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.num_false_positives(privileged=privileged))

    def num_instances(self, privileged=None):
        if privileged is None:
            return "Number of instances: {}".format(
                self.metric.num_instances(privileged=privileged))
        return "Number of {} instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.num_instances(privileged=privileged))

    def num_negatives(self, privileged=None):
        if privileged is None:
            return "Number of negative-outcome instances: {}".format(
                self.metric.num_negatives(privileged=privileged))
        return "Number of {} negative-outcome instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.num_negatives(privileged=privileged))

    def num_positives(self, privileged=None):
        if privileged is None:
            return "Number of positive-outcome instances: {}".format(
                self.metric.num_positives(privileged=privileged))
        return "Number of {} positive-outcome instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.num_positives(privileged=privileged))

    def num_pred_negatives(self, privileged=None):
        if privileged is None:
            return "Number of negative-outcome instances predicted: {}".format(
                self.metric.num_pred_negatives(privileged=privileged))
        return "Number of {} negative-outcome instances predicted: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.num_pred_negatives(privileged=privileged))

    def num_pred_positives(self, privileged=None):
        if privileged is None:
            return "Number of positive-outcome instances predicted: {}".format(
                self.metric.num_pred_positives(privileged=privileged))
        return "Number of {} positive-outcome instances predicted: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.num_pred_positives(privileged=privileged))

    def num_true_negatives(self, privileged=None):
        if privileged is None:
            return "Number of true negative instances (TN): {}".format(
                self.metric.num_true_negatives(privileged=privileged))
        return "Number of {} true negative instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.num_true_negatives(privileged=privileged))

    def num_true_positives(self, privileged=None):
        if privileged is None:
            return "Number of true positive instances (TP): {}".format(
                self.metric.num_true_positives(privileged=privileged))
        return "Number of {} true positive instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.num_true_positives(privileged=privileged))

    def positive_predictive_value(self, privileged=None):
        if privileged is None:
            return "Positive predictive value (PPV, precision = TP / (TP + FP)): {}".format(
                self.metric.positive_predictive_value(privileged=privileged))
        return "Positive predictive value on {} instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.positive_predictive_value(privileged=privileged))

    def statistical_parity_difference(self):
        return ("Statistical parity difference (probability of favorable "
                "outcome for unprivileged instances - probability of favorable "
                "outcome for privileged instances): {}".format(
                    self.metric.statistical_parity_difference()))

    def theil_index(self):
        return "Theil index (generalized entropy index with alpha = 1): {}".format(
            self.metric.theil_index())

    def true_negative_rate(self, privileged=None):
        if privileged is None:
            return "True negative rate (TNR, specificity = TN / (FP + TN)): {}".format(
                self.metric.true_negative_rate(privileged=privileged))
        return "True negative rate on {} instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.true_negative_rate(privileged=privileged))

    def true_positive_rate(self, privileged=None):
        if privileged is None:
            return "True positive rate (TPR, recall, sensitivity = TP / (TP + FN)): {}".format(
                self.metric.true_positive_rate(privileged=privileged))
        return "True positive rate on {} instances: {}".format(
            'privileged' if privileged else 'unprivileged',
            self.metric.true_positive_rate(privileged=privileged))

    def true_positive_rate_difference(self):
        return ("True positive rate difference (true positive rate on "
                "unprivileged instances - true positive rate on privileged "
                "instances): {}".format(
                    self.metric.true_positive_rate_difference()))

    # ============================== ALIASES ===================================
    def equal_opportunity_difference(self):
        return self.true_positive_rate_difference()

    def power(self, privileged=None):
        return self.num_true_positives(privileged=privileged)

    def precision(self, privileged=None):
        return self.positive_predictive_value(privileged=privileged)

    def recall(self, privileged=None):
        return self.true_positive_rate(privileged=privileged)

    def sensitivity(self, privileged=None):
        return self.true_positive_rate(privileged=privileged)

    def specificity(self, privileged=None):
        return self.true_negative_rate(privileged=privileged)


In [266]:
# Example:
from aif360.metrics import BinaryLabelDatasetMetric

# Calculating the disparate impact for Race
sens_ind = 0

# Printing the name of the sensitive attribute
print(f"{list(unprivileged_groups[sens_ind].keys())[0]}")

metric_race = BinaryLabelDatasetMetric(binary_dataset,
                                       privileged_groups=[
                                           privileged_groups[sens_ind]],
                                       unprivileged_groups=[
                                           unprivileged_groups[sens_ind]]
                                       )
metric_race


Race


In [267]:
# instantiate a metric text explainer using the BinaryLabelDatasetMetric object
explainer_race = MetricTextExplainer(metric_race)
explainer_race


In [268]:
# Calculate the disparate impact for the available protected attribute and instantiate the explainer
sens_ind = 0  # Use 0 since only one attribute is present in the groups lists
metric_sex = BinaryLabelDatasetMetric(
	binary_dataset,
	privileged_groups=[privileged_groups[sens_ind]],
	unprivileged_groups=[unprivileged_groups[sens_ind]]
)
explainer_sex = MetricTextExplainer(metric_sex)
explainer_sex


In [269]:
# Calculating the disparate impact for sex
sens_ind = 0  # Ensure this index is within the range of unprivileged_groups and privileged_groups
print(f"{list(unprivileged_groups[sens_ind].keys())[0]}")

# Instantiate the metric class for the protected attribute.
metric_sex = BinaryLabelDatasetMetric(binary_dataset,
                                  privileged_groups=[privileged_groups[sens_ind]],
                                  unprivileged_groups=[unprivileged_groups[sens_ind]]
                                  )

# Instantiate the explainer for each protected attribute
explainer_sex = MetricTextExplainer(metric_sex)


Race


## Fairness Metrics

- Disparate Impact
- Mean Difference
- Base Rate
- Statistical Parity Difference

### Disparate Impact

In [270]:
# Calculate the disparate impact for sex
print(f"Disparate Impact (Sex): {metric_sex.disparate_impact()}")

# Exlpaining the disparate impact
print(explainer_sex.disparate_impact())


Disparate Impact (Sex): 1.0213159658478306
Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 1.0213159658478306


In the case of Sex, since the value is greater than 1 (1.04) the dataset is biased, with Females being more likely to be favorable than male. 

In [ ]:
# Display the disparate impact for Race
print(f"Disparate Impact (Race): {metric_race.disparate_impact()}")
print(explainer_race.disparate_impact())


Disparate Impact (Race): 1.0213159658478306
Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 1.0213159658478306


In the case of Race, since the value is greater than 1 (1.04) the dataset is actually biased with non-whites being more likely to be favorable than male.

### Mean Difference

In [272]:
print(f"Mean Difference (Sex): {metric_sex.mean_difference()}")
print(explainer_sex.mean_difference())


Mean Difference (Sex): 0.010803187050944074
Mean difference (mean label value on unprivileged instances - mean label value on privileged instances): 0.010803187050944074


In [273]:
print(f"Mean Difference (Race): {metric_race.mean_difference()}")
print(explainer_race.mean_difference())


Mean Difference (Race): 0.010803187050944074
Mean difference (mean label value on unprivileged instances - mean label value on privileged instances): 0.010803187050944074


### Base Rate

In [274]:
# Base Rate
# print(explainer_train.base_rate())
priv_base_rate = metric_sex.base_rate()
unprov_base_rate = metric_sex.base_rate(privileged=False)


In [275]:
print("For protected attribute: Sex")
print(f"- Privileged Base Rate: {priv_base_rate}")
print(f"- Unprivileged Base Rate: {unprov_base_rate}")


For protected attribute: Sex
- Privileged Base Rate: 0.5160790391321194
- Unprivileged Base Rate: 0.5176151761517616


In [276]:
# Base Rate
# print(explainer_train.base_rate())
priv_base_rate = metric_race.base_rate()
unprov_base_rate = metric_race.base_rate(privileged=False)


In [277]:
print("For protected attribute: Race")
print(f"- Privileged Base Rate: {priv_base_rate}")
print(f"- Unprivileged Base Rate: {unprov_base_rate}")


For protected attribute: Race
- Privileged Base Rate: 0.5160790391321194
- Unprivileged Base Rate: 0.5176151761517616


### Statistical Parity Difference

In [278]:
# Statistical Parity Difference for Sex
print(
    f"Statistical Parity Difference (Sex): {metric_sex.statistical_parity_difference()}")


Statistical Parity Difference (Sex): 0.010803187050944074


In [279]:
print(
    f"Statistical Parity Difference (Race): {metric_race.statistical_parity_difference()}")


Statistical Parity Difference (Race): 0.010803187050944074


In [280]:
from sklearn.model_selection import train_test_split

dataset_orig_valid, dataset_orig_test = train_test_split(final_df, test_size=0.5, shuffle=True, random_state=42)


### Obtain scores from Test Set

In [ ]:
# dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
# X_test = scale_orig.transform(dataset_orig_test_pred.features)
# y_test = dataset_orig_test_pred.labels
# dataset_orig_test_pred.scores = lmod.predict_proba(X_test)[:, pos_ind].reshape(-1,1)


### Prediction from Test Set

In [ ]:
# # Metrics for the test set
# fav_inds = dataset_orig_test_pred.scores > best_class_thresh
# dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
# dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label

# display(Markdown("#### Test set"))
# display(Markdown(
#     "##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

# metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred,
#                                   unprivileged_groups, privileged_groups)


#### Test set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Balanced accuracy = 0.7437
Statistical parity difference = -0.3580
Disparate impact = 0.2794
Average odds difference = -0.3181
Equal opportunity difference = -0.3769
Theil index = 0.1129


In [ ]:
# # Metrics for the transformed test set
# dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)

# display(Markdown("#### Test set"))
# display(Markdown("##### Transformed predictions - With fairness constraints"))
# metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred,
#                                   unprivileged_groups, privileged_groups)


#### Test set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 0.7141
Statistical parity difference = -0.0402
Disparate impact = 0.9088
Average odds difference = 0.0423
Equal opportunity difference = 0.0407
Theil index = 0.1171
